### pdf process

In [17]:
# import packages
import PyPDF2
import re
import pandas as pd 
import numpy as np

In [18]:
# open the pdf file
reader = PyPDF2.PdfReader("fulltext/10457737.pdf")

# get number of pages
num_pages = len(reader.pages)
print(num_pages)

10


In [19]:
def DocContains(string, reader):
    for page in reader.pages:
        text = page.extract_text() 
        res_search = re.search(string, text)
        if res_search is not None:
            return True
    return False

In [20]:
DocContains("EQ-5D", reader)

True

### read database

In [21]:
# import pandas as pd
df = pd.read_csv("EQ5D_fulltext_PMID.tsv", sep="\t")

In [22]:
df.head()

,Doi,rev12,filter,filename
0,10.1186/s12955-021-01889-4,FY,1,34781978
1,10.1093/humrep/dex057,FY,2,28383700
2,10.1007/s11136-021-03007-9,N,1,34613597
3,10.1186/s12955-020-01475-0,AY,1,32727498
4,10.1007/s10198-021-01307-y,FY,1,33893889


In [23]:
tmp = df[df['filter']==1]
train_neg = tmp[tmp['rev12']=='N']['filename'].to_numpy()
train_pos = tmp[tmp['rev12']!='N']['filename'].to_numpy()
tmp = df[df['filter']==2]
test_neg = tmp[tmp['rev12']=='N']['filename'].to_numpy()
test_pos = tmp[tmp['rev12']!='N']['filename'].to_numpy()

In [24]:
len(train_pos), len(train_neg), len(test_pos), len(test_neg)

(61, 39, 60, 40)

In [25]:
fn = '10457737'
def getSubset(fn, df):
    tmp = df[df['filename']==fn]
    if tmp.shape[0] == 0:
        raise Exception("not found", fn)
    if tmp.shape[0] > 1:
        raise Exception("too many", [fn, tmp.shape[0]])
    fv = 'train' if tmp['filter'].to_numpy()[0] == 1 else 'test'
    rev = tmp['rev12'].to_numpy()[0] != 'N'
    return fv, rev

In [26]:
getSubset(fn, df)

('train', True)

### batch process all pdfs in dir

In [ ]:
import os
results = {}
Hal = 0;
idx = 1;
i = 0;
j = 0;
idx_Pdf_Name = 1;
list_headrow_1 = ["accuracy", "sensitivity", "precision", "specificity"]
list_kw_1 = ["index", "value", "utility", "utilities", "score", "mobility", "self-care", "usual activities", "pain", "discomfort", "anxiety", "depression", "looking after myself", "doing usual activities", "having pain or discomfort", "feeling worried", "feeling sad", "feeling unhappy"]
list_PDF_Name = []
# len (list_kw_1)

# Fill the PDF name into list_PDF_Name array
list_PDF_Name = {};
for fn in os.listdir('fulltext'):
    if not fn.endswith(".pdf"):
        continue
    reader = PyPDF2.PdfReader("fulltext/" + fn)
    list_PDF_Name[idx_Pdf_Name] = fn[:-4]
    idx_Pdf_Name += 1
    
# print(list_PDF_Name)
    
print (" ===============================================================================")

y = len(list_kw_1) + 1 # PDF name in first Column, and 18 parameter value
x = idx_Pdf_Name + 1 

print ("x: " , x, "   y: ", y , "\n")

Matrix = [[0 for x1 in range(x)] for y1 in range(y)]

print ("Matrix row number:", x , "Martix column number: " , y , "idx_Pdf-Name" , idx_Pdf_Name , "\n")

for i in range(idx_Pdf_Name-1):
    # print ("Sorsz: " , i+1, "    " , list_PDF_Name[i+1], "     " , idx_Pdf_Name-1 , "\n")
    Matrix[0][i+1] = list_PDF_Name[i+1]

    
print (np.matrix(Matrix))

for j in range(y-1):
    # print ("Sorsz: " , j, "    " , list_kw_1[j], "     " , y , "\n")
    Matrix[j+1][0] = list_kw_1[j]

print (np.matrix(Matrix))


# df_results = pd.MultiIndex(levels=[[list_PDF_Name],[list_kw_1]])
# df_results =pd. MultiIndex(columns = [list_headrow_1])


# print (df_results)

print (" ===============================================================================")

Mtx_Row = 1
Mtx_Col = 1


for fn in os.listdir('fulltext'):
    if not fn.endswith(".pdf"):
        continue
    reader = PyPDF2.PdfReader("fulltext/" + fn)
    arr = {};
    for kw in list_kw_1:
        arr[kw] = DocContains(kw, reader)
        print ("Row: ", Mtx_Col, " Column: " , Mtx_Row , " Arr: ", arr[kw] , "\n")
        Matrix[Mtx_Row][Mtx_Col] = arr[kw]
        Mtx_Row += 1
    # print(idx, fn[:-4], arr)
    idx += 1
    Mtx_Col += 1
    Mtx_Row = 1
    results[fn[:-4]] = arr
    

print (np.matrix(Matrix))

In [62]:
df = pd.DataFrame(Matrix)

In [63]:
#df = df.reset_index().drop('index', axis=1)
#df.drop(columns=df.columns[0], axis=1,  inplace=True) #drop first column
df.drop(columns=df.columns[-1], axis=1,  inplace=True) #drop last column
df.to_csv("fulltext_analysis_detailed.csv")